In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
launch_sites = spacex_df['Launch Site'].unique().tolist()
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(launch_sites)
# Create a dash application
app = dash.Dash(__name__)

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']


In [2]:
dropdown_options = (
    [{'label': 'All Sites', 'value': 'ALL'}] +
    [{'label': site, 'value': site} for site in launch_sites]
)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),

    html.Div(dcc.Dropdown(id='site-dropdown',
                options=dropdown_options,
                value='ALL',
                placeholder="Select a launch site",
                searchable=True
                )),

    html.Div(dcc.Graph(id='success-pie-chart')),

     html.Br(),
    dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                    marks={0: '0',
                           100: '100'},
                    value=[0, 10000]),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df,
                     values='class',
                     names='Launch Site',
                     title='Total Success Launches by site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        pie_data = filtered_df['class'].value_counts().reset_index()
        pie_data.columns = ['class', 'count']
        #pie_data['outcome'] = pie_data['class'].map({1: 'Success', 0: 'Failure'})
        fig = px.pie(
            pie_data,
            names='class',
            values='count',
            title=f'Total Success vs Failure for {entered_site}'
        )
    return fig



@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range

    # Filter data by payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    if selected_site == 'ALL':
        # Show all sites with filtered payload range
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Correlation Between Payload and Success for All Sites'
        )
    else:
        # Filter by selected site
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Correlation Between Payload and Success for {selected_site}'
        )

    return fig
if __name__ == '__main__':
    app.run()



Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/farjn/PycharmProjects/ibm_data_science/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

